In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fastf1 as ff1

ff1.Cache.enable_cache('cache')

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
schedule2022 = ff1.get_event_schedule(2022, include_testing=False)

In [4]:
schedule2022.head()

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session2,Session2Date,Session3,Session3Date,Session4,Session4Date,Session5,Session5Date,F1ApiSupport
2,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2022,2022-03-20 20:00:00,Bahrain Grand Prix,conventional,Practice 1,2022-03-18 15:00:00,Practice 2,2022-03-18 18:00:00,Practice 3,2022-03-19 15:00:00,Qualifying,2022-03-19 18:00:00,Race,2022-03-20 18:00:00,True
3,2,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2022,2022-03-27 22:00:00,Saudi Arabian Grand Prix,conventional,Practice 1,2022-03-25 17:00:00,Practice 2,2022-03-25 20:00:00,Practice 3,2022-03-26 17:00:00,Qualifying,2022-03-26 20:00:00,Race,2022-03-27 20:00:00,True
4,3,Australia,Melbourne,FORMULA 1 HEINEKEN AUSTRALIAN GRAND PRIX 2022,2022-04-10 17:00:00,Australian Grand Prix,conventional,Practice 1,2022-04-08 13:00:00,Practice 2,2022-04-08 16:00:00,Practice 3,2022-04-09 13:00:00,Qualifying,2022-04-09 16:00:00,Race,2022-04-10 15:00:00,True
5,4,Italy,Imola,FORMULA 1 ROLEX GRAN PREMIO DEL MADE IN ITALY ...,2022-04-24 17:00:00,Emilia Romagna Grand Prix,sprint,Practice 1,2022-04-22 13:30:00,Qualifying,2022-04-22 17:00:00,Practice 2,2022-04-23 12:30:00,Sprint,2022-04-23 16:30:00,Race,2022-04-24 15:00:00,True
6,5,United States,Miami,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2022,2022-05-08 17:30:00,Miami Grand Prix,conventional,Practice 1,2022-05-06 14:30:00,Practice 2,2022-05-06 17:30:00,Practice 3,2022-05-07 13:00:00,Qualifying,2022-05-07 16:00:00,Race,2022-05-08 15:30:00,True


In [5]:
def get_telemetry_at_start_of_lap(lap, telemetry):
    mask = (telemetry['Date'] > lap['LapStartDate']) & (telemetry['Date'] <= lap['LapStartDate'] + pd.Timedelta(seconds=1))
    row = telemetry.loc[mask].iloc[0]
    telemetryInfo = row[['DriverAhead', 'DistanceToDriverAhead']]
    lapInfo = lap[['LapNumber', 'LapTime', 'Compound', 'TyreLife', 'Stint', 'TrackStatus']]
    telemetryInfo.rename("Telemetry", inplace=True)
    lapInfo.rename("Lap", inplace=True)
    merge = pd.concat([telemetryInfo, lapInfo])
    return merge

In [6]:
class NoLapException(Exception):
    pass

def get_laps_of_driver(driver, laps):
    driver_laps = laps.pick_driver(driver)
    if len(driver_laps['DriverNumber']) == 0:
        raise NoLapException("No laps for driver " + driver)
    driver_laps_telemetry = driver_laps.get_car_data().add_driver_ahead()
    transformed_laps = []
    for index, row in driver_laps.iterrows():
        transformed_laps.append(get_telemetry_at_start_of_lap(row, driver_laps_telemetry))
    return pd.DataFrame(transformed_laps)

In [7]:
for index, event in schedule2022.iterrows():
    print("Processing race round - ", event['RoundNumber'])
    race = event.get_race()
    race.load()
    df = pd.DataFrame(
        columns=[
            'LapNumber',
            'LapTime',
            'Compound',
            'TyreLife',
            'TrackStatus',
            'Stint',
            'DistanceToDriverAhead',
            'DriverAhead',
            ])
    for driver in race.drivers:
        print("     Processing driver - ", driver)
        try:
            df = pd.merge(df, get_laps_of_driver(driver, race.laps), how='outer')
        except NoLapException as e:
            print(e)
    
    # Save it to a csv file
    df.to_csv('data/'+ event.EventName +' .csv')

core           INFO 	Loading data for Bahrain Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info


Processing race round -  1


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '55', '44', '63', '20', '77', '31', '22', '14', '24', '47', '18', '23', '3', '4', '6', '27', '11', '1', '10']


     Processing driver -  16
     Processing driver -  55
     Processing driver -  44
     Processing driver -  63
     Processing driver -  20
     Processing driver -  77
     Processing driver -  31
     Processing driver -  22
     Processing driver -  14
     Processing driver -  24
     Processing driver -  47
     Processing driver -  18
     Processing driver -  23
     Processing driver -  3
     Processing driver -  4
     Processing driver -  6
     Processing driver -  27
     Processing driver -  11
     Processing driver -  1
     Processing driver -  10


core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info


Processing race round -  2


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
core        WARNING 	No lap data for driver 22
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 19 drivers: ['1', '16', '55', '11', '63', '31', '4', '10', '20', '44', '24', '27', '18', '23', '77', '14', '3', '6', '22']


     Processing driver -  1
     Processing driver -  16
     Processing driver -  55
     Processing driver -  11
     Processing driver -  63
     Processing driver -  31
     Processing driver -  4
     Processing driver -  10
     Processing driver -  20
     Processing driver -  44
     Processing driver -  24
     Processing driver -  27
     Processing driver -  18
     Processing driver -  23
     Processing driver -  77
     Processing driver -  14
     Processing driver -  3
     Processing driver -  6
     Processing driver -  22


NoLapException: No laps for driver 22